## MNIST Handwritten Digit dataset
![alt text](https://www.researchgate.net/profile/Steven_Young11/publication/306056875/figure/fig1/AS:393921575309346@1470929630835/Example-images-from-the-MNIST-dataset.png)

In [ ]:
from sklearn import datasets

digits = datasets.load_digits()
X_digits = digits.data
y_digits = digits.target
print(X_digits.shape)
print(y_digits)

(1797, 64)
[0 1 2 ... 8 9 8]


#1. เขียนคำสั่งแบ่งชุดข้อมูลสอนและข้อมูลทดสอบเป็น 80% และ 20% ตามลำดับ

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X_digits, y_digits, test_size=0.2)
print(X_test.shape)

(360, 64)


## 2. สร้างโมเดล 1-NN โดยใช้ข้อมูลสอนที่ถูกแบ่งไว้ก่อนหน้า จากนั้นทดสอบด้วยข้อมูลทดสอบ และแสดงค่าความแม่นยำ (accuracy)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

model =  KNeighborsClassifier(n_neighbors = 1)
model.fit(X_train, Y_train)

result1 = model.score(X_test, Y_test)
print('Accuracy %.2f%%' % (result1*100.0))

Accuracy 99.44%


## 3. เขียนคำสั่ง normalize ค่าข้อมูลก่อนนำไปฝึกฝน 1-NN เปรียบเทียบผลลัพธ์ระหว่างการ normalize และ non-normalize อินพุตข้อมูล ว่าความแม่นยำแบบไหนสูงกว่ากัน

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX1 = scaler.fit_transform(X_train)             # normalize training dataset into 0 to 1
rescaledX2 = scaler.fit_transform(X_test)              # normalize test dataset into 0 to 1

model =  KNeighborsClassifier(n_neighbors = 1)
model.fit(rescaledX1, Y_train)

result2 = model.score(rescaledX2, Y_test)
print("Accuracy %.2f%%" % (result2*100.0))   # print accuracy of KNN

Accuracy 99.44%


In [ ]:
print(X_test)
print(rescaledX2)

[[ 0.  0. 11. ...  1.  0.  0.]
 [ 0.  0.  4. ...  8.  0.  0.]
 [ 0.  0.  0. ... 15.  4.  0.]
 ...
 [ 0.  3. 11. ...  0.  0.  0.]
 [ 0.  0.  7. ...  3.  0.  0.]
 [ 0.  0.  5. ... 14.  5.  0.]]
[[0.     0.     0.6875 ... 0.0625 0.     0.    ]
 [0.     0.     0.25   ... 0.5    0.     0.    ]
 [0.     0.     0.     ... 0.9375 0.25   0.    ]
 ...
 [0.     0.6    0.6875 ... 0.     0.     0.    ]
 [0.     0.     0.4375 ... 0.1875 0.     0.    ]
 [0.     0.     0.3125 ... 0.875  0.3125 0.    ]]


##  4. เขียนคำสั่งแสดงค่าความแม่นยำสำหรับ k-NN  โดยที่ k = 1, 3, 5, 7, 9, ...., 21

In [ ]:
for i in range(1, 23, 2):
    model =  KNeighborsClassifier(n_neighbors = i)
    model.fit(X_train, Y_train)
    result = model.score(X_test, Y_test)
    print('K= ', i, "Accuracy %.2f%%" % (result*100.0))   # print accuracy of KNN

K=  1 Accuracy 99.44%
K=  3 Accuracy 98.61%
K=  5 Accuracy 99.17%
K=  7 Accuracy 98.33%
K=  9 Accuracy 98.61%
K=  11 Accuracy 98.33%
K=  13 Accuracy 97.78%
K=  15 Accuracy 97.22%
K=  17 Accuracy 97.50%
K=  19 Accuracy 97.50%
K=  21 Accuracy 97.50%


## 5.เขียนคำสั่งสร้างโมเดล Nearest Centroid โดยใช้ข้อมูล normalize แล้ว พร้อมเปรียบเทียบผลลัพธ์กับ 1-NN   

In [ ]:
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier

model_knn =  KNeighborsClassifier(n_neighbors = 1)
model_centr = NearestCentroid()

model_knn.fit(rescaledX1, Y_train)
model_centr.fit(rescaledX1, Y_train)

result1 = model_knn.score(rescaledX2, Y_test)
result2 = model_centr.score(rescaledX2, Y_test)

print("Accuracy of 1-nn : {} , Accuracy of nearest centroid {}".format(result1*100.0, result2*100.0))   # print accuracy of KNN



Accuracy of 1-nn : 99.44444444444444 , Accuracy of nearest centroid 91.94444444444444


## 4. ศึกษาอัลกอริทึม KD-tree จากเว็บ แล้วทำการปรับปรุงโค๊ดให้ใช้อัลกอริทึมดังกล่าว

http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html

In [ ]:
import numpy as np
from sklearn.neighbors import KDTree

np.random.seed(0)
tree = KDTree(X_train)         ## build kd-tree

idx = 5
nearest_dist, nearest_ind = tree.query([X_test[idx,:]], k = 1)    # k=1 nearest neighbors
print('nearest sample : {} with distance {}'.format(nearest_ind[0], nearest_dist[0]))

print('predicted label = ', Y_train[nearest_ind[0,0]], ' actual label : ', Y_test[idx])

nearest sample : [250] with distance [11.5758369]
predicted label =  0  actual label :  0


In [ ]:
import numpy as np
from sklearn.neighbors import KDTree

np.random.seed(0)
tree = KDTree(X_train)

nearest_dist, nearest_ind = tree.query(X_test, k = 1)  # k=1 nearest neighbors

#print(nearest_ind)

KK = Y_train[nearest_ind[:,0]]
print('actual label ', Y_test)

print('Accuracy = ', (1.0 - np.sum(KK != Y_test)/len(KK))*100)

actual label  [7 7 0 5 4 0 6 8 6 3 7 7 0 7 0 7 0 3 3 7 4 6 9 6 6 4 8 4 6 8 8 6 5 5 2 2 9
 4 9 9 7 9 0 5 8 6 0 1 5 8 1 4 0 2 5 0 5 9 9 3 7 2 3 8 6 3 6 7 4 6 6 9 6 2
 9 9 6 4 6 6 5 1 4 1 8 5 2 0 9 9 1 3 3 1 0 0 6 2 4 6 5 8 7 3 4 0 6 8 2 6 1
 6 9 6 6 3 6 5 2 2 2 0 7 9 9 3 3 5 3 7 9 0 3 4 3 1 5 5 1 6 0 6 8 2 3 7 6 4
 4 8 1 8 5 8 8 9 1 5 4 0 8 2 0 1 1 5 5 0 8 7 2 9 3 4 1 8 1 2 3 1 6 8 3 4 9
 0 7 5 5 9 7 0 3 3 5 7 9 6 4 5 2 6 3 1 4 1 1 5 7 5 0 7 0 1 9 2 4 2 2 2 0 3
 3 7 6 7 9 7 8 4 4 2 2 6 5 7 2 9 3 6 8 9 0 7 2 5 8 3 0 7 9 6 2 5 9 7 1 2 0
 4 3 5 7 9 2 6 3 0 0 0 8 9 8 1 5 4 2 6 4 7 5 6 6 3 2 2 1 5 7 1 2 5 4 5 9 7
 4 6 8 4 2 5 0 9 9 1 6 5 6 5 5 4 8 0 0 9 8 3 8 6 8 3 1 4 3 7 8 9 8 6 4 7 5
 3 1 0 9 1 8 8 1 9 9 7 2 7 5 7 8 0 1 8 4 6 0 7 6 2 8 1]
Accuracy =  98.88888888888889
